In [ ]:
# Load the full DataFrame
with open('preprocessed_data.pkl', 'rb') as f:
    df = pickle.load(f)

In [34]:
import pandas as pd
import torch
import numpy as np
from tensorly.decomposition import parafac
from tensorly.tenalg import mode_dot
import tensorly as tl

# Make sure TensorLy uses PyTorch backend
tl.set_backend('pytorch')

# Your tensor decomposition function
def tensor_decomp(X, rank=10):
    factors = parafac(X, rank=rank)
    _, _, C = factors.factors  # Get third factor matrix
    P = torch.linalg.pinv(C)   # Compute pseudo-inverse of C
    X_reduced = mode_dot(X, P, mode=2)  # Project X along mode-2
    return X_reduced

# Apply the function to each spectrogram in your DataFrame
reduced_spectrograms = []

for i, row in df.iterrows():
    spectrogram = row['spectrogram']
    
    # Convert spectrogram to torch tensor
    tensor = torch.tensor(spectrogram, dtype=torch.float32)
    
    # Decompose and reduce
    try:
        reduced = tensor_decomp(tensor, rank=10)
        reduced_spectrograms.append(reduced.numpy())  # convert back to NumPy for storing
    except Exception as e:
        print(f"Error on index {i}: {e}")
        reduced_spectrograms.append(None)  # Handle failure case

# Add reduced spectrograms to DataFrame
df['reduced_spectrogram'] = reduced_spectrograms

In [37]:
df.to_pickle('decomposed_data.pkl')

In [41]:
df['spectrogram'][0].shape

(119, 65, 28)

In [43]:
df['reduced_spectrogram'][0].shape

(119, 65, 10)